<a href="https://colab.research.google.com/github/Nishasathish13/TheSchoolofAI-END3.0/blob/main/Session%204%20-%20RNN%20are%20dead%2C%20and%20HandsOn/Class%20Notes/END3_Session_4_RNN_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

#to import readymade data: deprecated
from torchtext.legacy import data

#to ensure repeatability
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#using spacy tokenization method
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)

In [2]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 15.6MB/s]


In [3]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [5]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [6]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [7]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
# output = 25002, the 2 extra are unknown[unk] and pad. also end, start, class
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [8]:
#most common 20 words
print(TEXT.vocab.freqs.most_common(20))

[('the', 202755), (',', 192304), ('.', 166205), ('and', 109267), ('a', 108951), ('of', 100755), ('to', 93415), ('is', 76409), ('in', 61231), ('I', 54331), ('it', 53702), ('that', 49281), ('"', 44129), ("'s", 43421), ('this', 42424), ('-', 37156), ('/><br', 35492), ('was', 35161), ('as', 30491), ('with', 29823)]


In [10]:
#when we say batch_size=X, we create X instances of the same model. The 1st word of the 64 sentences are processed in 64 instances at a time
BATCH_SIZE = 64

#use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#will not use iterator in transformers
#bucketiterator sorts the sentences according to the number of words, creates container of senstenses with similar number of words and also shuffles them within each container.
#This is done in order to avoid a lot of padding, so number of computes are reduced. 
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [11]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        #embedding dim determines how good our network is (usually set to 512)!
        #Money, compute power, memory determines the embedding_dim
        #input dim are the total number of words!
        #the embeddings are initialized randomly, and then they learn to represent each word in the context of our sentences/Vocab.
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [12]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [13]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

#we have a model of 2.5 million trainable params

The model has 2,592,105 trainable parameters


In [17]:
import torch.optim as optim

#Optimizer = SDG to update the gradients
optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [18]:
#defining our loss
criterion = nn.BCEWithLogitsLoss()

In [19]:
model = model.to(device)
criterion = criterion.to(device)

In [20]:
#binary accuracy as we have two classes
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [21]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    #calculating for each batch of the iterator
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [22]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [24]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 10m 6s
	Train Loss: 0.694 | Train Acc: 50.27%
	 Val. Loss: 0.697 |  Val. Acc: 49.24%
Epoch: 02 | Epoch Time: 10m 0s
	Train Loss: 0.693 | Train Acc: 50.14%
	 Val. Loss: 0.697 |  Val. Acc: 49.49%
Epoch: 03 | Epoch Time: 10m 4s
	Train Loss: 0.693 | Train Acc: 49.85%
	 Val. Loss: 0.697 |  Val. Acc: 50.83%
Epoch: 04 | Epoch Time: 10m 5s
	Train Loss: 0.693 | Train Acc: 49.93%
	 Val. Loss: 0.697 |  Val. Acc: 48.57%
Epoch: 05 | Epoch Time: 10m 4s
	Train Loss: 0.693 | Train Acc: 50.17%
	 Val. Loss: 0.697 |  Val. Acc: 50.77%
